In [75]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.metrics import make_scorer
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

In [69]:
train = pd.read_csv('open/train.csv').drop(columns=['ID'])
test = pd.read_csv('open/test.csv').drop(columns=['ID'])

In [70]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']
X = X.drop(columns=['임신 시도 또는 마지막 임신 경과 연수', '착상 전 유전 검사 사용 여부', 'PGD 시술 여부','PGS 시술 여부','난자 해동 경과일','배아 해동 경과일'])
test = test.drop(columns=['임신 시도 또는 마지막 임신 경과 연수', '착상 전 유전 검사 사용 여부', 'PGD 시술 여부','PGS 시술 여부','난자 해동 경과일','배아 해동 경과일'])
#임신 시도 또는 마지막 임신 경과 연수 열 삭제
#착상 전 유전 검사 사용 여부 삭제
#PGD 시술 여부 삭제
#PGS 시술 여부 삭제
#난자 해동 경과일 삭제
#배아 해동 경과일 삭제

In [71]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부"
]

# One-Hot Encoder 초기화
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# 인코딩 적용
X_train_encoded = onehot_encoder.fit_transform(X[categorical_columns])
X_test_encoded = onehot_encoder.transform(test[categorical_columns])

# 인코딩된 데이터를 데이터프레임으로 변환
X_train_encoded = pd.DataFrame(X_train_encoded, columns=onehot_encoder.get_feature_names_out(categorical_columns))
X_test_encoded = pd.DataFrame(X_test_encoded, columns=onehot_encoder.get_feature_names_out(categorical_columns))

# 기존 데이터프레임과 결합
X_train_encoded = pd.concat([X.drop(columns=categorical_columns).reset_index(drop=True), X_train_encoded], axis=1)
X_test_encoded = pd.concat([test.drop(columns=categorical_columns).reset_index(drop=True), X_test_encoded], axis=1)

In [72]:
numeric_columns = [
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
]

X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)
y=y.fillna(0)

X_train_encoded.columns = X_train_encoded.columns.str.replace(r'[\[\]{}":,]', '', regex=True)
X_test_encoded.columns = X_test_encoded.columns.str.replace(r'[\[\]{}":,]', '', regex=True)
X_train_encoded.columns = X_train_encoded.columns.str.replace(" ", "_")
X_test_encoded.columns = X_test_encoded.columns.str.replace(" ", "_")

In [ ]:
# 목적 함수 정의 (Optuna가 최소화할 함수)
def objective(trial):
    # 튜닝할 하이퍼파라미터 정의
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 5, 30),
        "num_leaves": trial.suggest_int("num_leaves", 50, 500),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 30),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.8, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "class_weight": "balanced",  # 클래스 불균형 고려
        "random_state": 101
    }

    # 모델 생성
    model = LGBMClassifier(**params)

    # Stratified K-Fold 설정 (클래스 비율 유지)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=101)

    # 5-Fold 교차검증으로 ROC-AUC 점수 계산
    roc_auc_scores = cross_val_score(model, X_train_encoded, y, cv=skf, scoring="roc_auc")
    
    return roc_auc_scores.mean()  # 평균 ROC-AUC 반환 (최대화 목표)

# Optuna 스터디 생성 및 최적화 수행
study = optuna.create_study(direction="maximize")  # ROC-AUC를 최대화
study.optimize(objective, n_trials=50)  # 50번 반복해서 최적의 하이퍼파라미터 찾기

# 최적의 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)
print("Best ROC-AUC:", study.best_value)


In [60]:
model = LGBMClassifier(class_weight="balanced", random_state=101)
model.fit(X_train_encoded, y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 793, number of negative: 2397
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001266 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 532
[LightGBM] [Info] Number of data points in the train set: 3190, number of used features: 139
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


LGBMClassifier(class_weight='balanced', random_state=101)

In [61]:
# Stratified K-Fold 설정 (클래스 비율 유지)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=101)

# 5-Fold 교차 검증 수행 (ROC-AUC 기준)
roc_auc_scores = cross_val_score(model, X_train_encoded, y, cv=skf, scoring="roc_auc")

# 결과 출력
print("각 Fold의 ROC-AUC:", roc_auc_scores)
print("평균 ROC-AUC:", roc_auc_scores.mean())

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 635, number of negative: 1917
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001175 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 519
[LightGBM] [Info] Number of data points in the train set: 2552, number of used features: 135
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 635, number of negative: 1917
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001159 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise

In [62]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

sample_submission = pd.read_csv('open/sample_submission.csv')
sample_submission['probability'] = pred_proba

sample_submission.to_csv('./baseline_submit.csv', index=False)